In [1]:
import pandas as pd

avaliacoes = {
    'Ana': {
        'Freddy x Jason': 2.5, 
        'O Ultimato Bourne': 3.5,
        'Star Trek': 3.0, 
        'Exterminador do Futuro': 3.5, 
        'Norbit': 2.5, 
        'Star Wars': 3.0
    },
    'Marcos': {
        'Freddy x Jason': 3.0, 
        'O Ultimato Bourne': 3.5, 
        'Star Trek': 1.5, 
        'Exterminador do Futuro': 5.0, 
        'Star Wars': 3.0, 
        'Norbit': 3.5
    },
    'Pedro': {
        'Freddy x Jason': 2.5, 
        'O Ultimato Bourne': 3.0,
        'Exterminador do Futuro': 3.5, 
        'Star Wars': 4.0
    },
    'Claudia': {
        'O Ultimato Bourne': 3.5, 
        'Star Trek': 3.0,
        'Star Wars': 4.5, 
        'Exterminador do Futuro': 4.0, 
        'Norbit': 2.5
    },
    'Adriano': {
        'Freddy x Jason': 3.0, 
        'O Ultimato Bourne': 4.0, 
        'Star Trek': 2.0, 
        'Exterminador do Futuro': 3.0, 
        'Star Wars': 3.0,
        'Norbit': 2.0
    },
    'Janaina': {
        'Freddy x Jason': 3.0, 
        'O Ultimato Bourne': 4.0,
        'Star Wars': 3.0, 
        'Exterminador do Futuro': 5.0, 
        'Norbit': 3.5
    },
    'Leonardo': {
        'O Ultimato Bourne': 4.5,
        'Norbit': 1.0,
        'Exterminador do Futuro': 4.0
    }
}

df = pd.DataFrame(avaliacoes)
df

,Ana,Marcos,Pedro,Claudia,Adriano,Janaina,Leonardo
Freddy x Jason,2.5,3.0,2.5,NaN,3.0,3.0,NaN
O Ultimato Bourne,3.5,3.5,3.0,3.5,4.0,4.0,4.5
Star Trek,3.0,1.5,NaN,3.0,2.0,NaN,NaN
Exterminador do Futuro,3.5,5.0,3.5,4.0,3.0,5.0,4.0
Norbit,2.5,3.5,NaN,2.5,2.0,3.5,1.0
Star Wars,3.0,3.0,4.0,4.5,3.0,3.0,NaN


In [2]:
import math

def euclidiana(data, user1, user2):
    # Encontrar itens com avaliações não nulas para ambos os usuários
    common_items = data.columns[(~data.loc[user1].isna()) & (~data.loc[user2].isna())]

    if len(common_items) == 0:
        return 0

    # Calcular a similaridade euclidiana usando operações do Pandas
    squared_diff = (data.loc[user1, common_items] - data.loc[user2, common_items])**2
    euclidean_distance = math.sqrt(squared_diff.sum())

    # Retornar a similaridade normalizada
    return 1 / (1 + euclidean_distance)

In [3]:
def get_similaridade(data, user):
    # Calcula a similaridade euclidiana entre o usuário dado e todos os outros usuários
    similarity_list = [(euclidiana(data, user, other), other)
                       for other in data.index if other != user]

    # Ordena a lista com base na similaridade em ordem decrescente
    similarity_list.sort(reverse=True)

    # Retorna os 30 usuários mais similares (ou todos, se houver menos de 30)
    return similarity_list[:min(len(similarity_list), 30)]

In [4]:
get_similaridade(df, 'Star Trek')

[(0.3483314773547883, 'Freddy x Jason'),
 (0.32037724101704074, 'Norbit'),
 (0.2989350844248255, 'Star Wars'),
 (0.2553967929896867, 'O Ultimato Bourne'),
 (0.20799159651347807, 'Exterminador do Futuro')]

In [9]:
def calcula_similares(Data):
    result = {}
    for filme in Data.index:
        notas = get_similaridade(Data, filme)
        result[filme] = notas

    return result

In [10]:
itens_similares = calcula_similares(df)
itens_similares

{'Freddy x Jason': [(0.4494897427831781, 'Norbit'),
  (0.38742588672279304, 'Star Wars'),
  (0.3483314773547883, 'Star Trek'),
  (0.3483314773547883, 'O Ultimato Bourne'),
  (0.2402530733520421, 'Exterminador do Futuro')],
 'O Ultimato Bourne': [(0.3483314773547883, 'Freddy x Jason'),
  (0.32037724101704074, 'Star Wars'),
  (0.3090169943749474, 'Exterminador do Futuro'),
  (0.2553967929896867, 'Star Trek'),
  (0.1886378647726465, 'Norbit')],
 'Star Trek': [(0.3483314773547883, 'Freddy x Jason'),
  (0.32037724101704074, 'Norbit'),
  (0.2989350844248255, 'Star Wars'),
  (0.2553967929896867, 'O Ultimato Bourne'),
  (0.20799159651347807, 'Exterminador do Futuro')],
 'Exterminador do Futuro': [(0.3090169943749474, 'O Ultimato Bourne'),
  (0.252650308587072, 'Star Wars'),
  (0.2402530733520421, 'Freddy x Jason'),
  (0.20799159651347807, 'Star Trek'),
  (0.1918253663634734, 'Norbit')],
 'Norbit': [(0.4494897427831781, 'Freddy x Jason'),
  (0.32037724101704074, 'Star Trek'),
  (0.2942980550855

In [11]:
def get_recomendacao_usuario(Data, usuario, nota_minima=1.0):
    similaridades = []

    # Calcular similaridade com todos os outros usuários
    for outro_usuario in Data.index:
        if outro_usuario == usuario:
            continue

        similaridade = euclidiana(Data, usuario, outro_usuario)
        similaridades.append((outro_usuario, similaridade))

    # Ordenar por similaridade em ordem decrescente
    similaridades.sort(key=lambda x: x[1], reverse=True)

    # Escolher o usuário mais similar (o primeiro da lista)
    usuario_mais_similar = similaridades[0][0]

    # Filtrar os filmes que o usuário mais similar assistiu e o usuário atual ainda não assistiu, considerando a nota mínima
    filmes_recomendados = Data.loc[usuario_mais_similar][
        (Data.loc[usuario].isna()) & (Data.loc[usuario_mais_similar] >= nota_minima)
    ].index.tolist()

    
    return filmes_recomendados[:30]

In [12]:
data = df.T
data

,Freddy x Jason,O Ultimato Bourne,Star Trek,Exterminador do Futuro,Norbit,Star Wars
Ana,2.5,3.5,3.0,3.5,2.5,3.0
Marcos,3.0,3.5,1.5,5.0,3.5,3.0
Pedro,2.5,3.0,NaN,3.5,NaN,4.0
Claudia,NaN,3.5,3.0,4.0,2.5,4.5
Adriano,3.0,4.0,2.0,3.0,2.0,3.0
Janaina,3.0,4.0,NaN,5.0,3.5,3.0
Leonardo,NaN,4.5,NaN,4.0,1.0,NaN


In [13]:
get_recomendacao_usuario(data, 'Leonardo')

['Freddy x Jason', 'Star Trek', 'Star Wars']

In [25]:
def get_recomendacoes_itens(Data, itens_similares, usuario):
    notas_usuario = Data[usuario]
    notas = {}
    total_similaridade = {}

    for filme, nota in notas_usuario.items():
        if nota >= 0:
            for similaridade, filme2 in itens_similares[filme]:
                if filme2 not in notas_usuario:
                    continue
                notas.setdefault(filme2, 0)
                notas[filme2] += similaridade * nota
                total_similaridade.setdefault(filme2, 0)
                total_similaridade[filme2] += similaridade
    ranking = [(score / total_similaridade[filme], filme) for filme, score in notas.items()]
    ranking.sort(reverse=True)

    return ranking[:10]

In [26]:
get_recomendacoes_itens(df, itens_similares, 'Leonardo')

[(4.247905512721092, 'Norbit'),
 (3.1667425234070894, 'Star Wars'),
 (3.1594808368799123, 'Exterminador do Futuro'),
 (2.936629402844435, 'Star Trek'),
 (2.868767392626467, 'Freddy x Jason'),
 (2.8628392069008184, 'O Ultimato Bourne')]